In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt  # doctest: +SKIP
from sklearn.metrics import plot_confusion_matrix
from sklearn.svm import SVC

In [2]:
features = ['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received']

def data_split(dataSet) :
    dataSet_X = dataSet.drop('No-show', axis = 1)
    dataSet_y = dataSet.drop(features, axis = 1)
    return dataSet_X, dataSet_y

In [3]:
# funzione che permette di dividere l'età in gruppi
def fascia_eta(dataSet):
    bins= [0,31,56,80,200]
    labels = [0, 1, 2, 3]
    
    dataSet['AgeGroup'] = pd.cut(dataSet['Age'], bins=bins, labels=labels, right=False)

In [4]:
# funzione che effettua la differenza tra le due date e riporta il risultato in millisecondi
def sottrai_data (row) :
    row['diffData'] = ((row.AppointmentDay - row.ScheduledDay).total_seconds() * 1000)
    return row

In [5]:
# funzione che permette di dividere la diff tra le due date in gruppi
def fascia_diffData(dataSet):
    bins= [-5.681600e+09, -2.987600e+07, 2.995910e+08, 1.238590e+09, 1.542714e+11]
    labels = [0, 1, 2, 3]
    
    dataSet['FasciadiffData'] = pd.cut(dataSet['diffData'], bins=bins, labels=labels, right=False)

In [6]:
# encoder che permette di mappare i valori in numeri così che sia più facile lavorarci
def enc_features(dataSet) :
    encoder = LabelEncoder()
    encoderColumns = ['PatientId', 'Gender', 'Neighbourhood', 'AppointmentDay']

    encoded = dataSet[encoderColumns].apply(encoder.fit_transform)
    
    dataSet = dataSet.drop('PatientId', axis = 1).drop('Gender', axis = 1).drop('Neighbourhood', axis = 1)
    dataSet = dataSet.drop('AppointmentDay', axis = 1)
    
    dataSet = dataSet.join(encoded)
    
    return dataSet


In [7]:
# path dei due file csv anche se inutile...zucchero sintattico
train_path = "train.csv"
test_path = "test.csv"

In [8]:
# carico i dati parsando le date
train = pd.read_csv(train_path, parse_dates=['ScheduledDay','AppointmentDay'], index_col=0)
test = pd.read_csv(test_path, parse_dates=['ScheduledDay','AppointmentDay'], index_col=0)

In [9]:
# divido le features dalla variabile obiettivo ('No-show')
train_X, train_y = data_split(train)
test_X, test_y = data_split(test)

In [10]:
# controllo che non esistano valori esplicitamente null nel trainSet
missing_val_count_by_column = (train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Series([], dtype: int64)


In [11]:
# divido le età in 4 fasce
fascia_eta(train_X)
fascia_eta(test_X)

In [ ]:
# effettuo la differenza tra la data della visita e quella del giorno di prenotazione
# magari qualcuno non si è presentato perchè ha prenotato troppo tempo prima e si è dimenticato...boh
train_X = train_X.apply(sottrai_data, axis = 'columns')
test_X = test_X.apply(sottrai_data, axis = 'columns')

In [ ]:
train_X.head()

In [ ]:
# features encoder per le seguenti colonne: ['PatientId', 'Gender', 'Neighbourhood', 'AppointmentDay']

train_X = enc_features(train_X)
test_X = enc_features(test_X)

In [ ]:
train_X.head()

In [ ]:
# divido in gruppi la diff di data così che il gruppo con numero più alto abbia anche prenotazione più lontana
fascia_diffData(train_X)
fascia_diffData(test_X)

In [ ]:
train_X.head()

In [ ]:
# per ogni colonna stampo i valori univoci in modo da assicurarmi che non ci siano valori strani
for col in train_X.columns :
    print("---", col, "---")
    print(train_X[col].unique())
    print("\n")

In [ ]:
train_X.columns

In [ ]:
# sono molto indeciso tra quali features usare, ma scarterò: 
# 'AppointmentID' perchè univoco;
# 'Age' perchè ormai ho 'AgeGroup'; 
# 'PatientId' perchè anche se magari non si è presentato una volta, non credo ci sia una correlazione così forte con 'No-show'; 
# 'ScheduledDay' perchè tutti valori univoci avendo anche i millisecondi nel timestamp (avevo anche provato a dividerli in mese e giorno, ma il modello si overfitta); 'AppointmentDay'  
# 'AppointmentDay' perchè ho usato la diff tra visita e prenotazione
# 'Neighbourhood' non credo ci sia una correlazione tra quartiere e persone che non si fanno visitare

cols = ['Scholarship', 'Hipertension', 'AgeGroup',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received',
       'Gender', 'FasciadiffData']

In [ ]:
# 42 because that is the meaning of life
clf = DecisionTreeClassifier(random_state=42)
# Train Decision Tree Classifer
clf = clf.fit(train_X[cols] , train_y)

print("Accuracy on training set: {:.3f}".format(clf.score(train_X[cols], train_y)))
print("Accuracy on test set: {:.3f}".format(clf.score(test_X[cols], test_y)))

In [ ]:
# plot che mi dice quanti valori sono stati classificati correttamente e non

plot_confusion_matrix(clf, test_X[cols], test_y)  # doctest: +SKIP
plt.show()